# This notebook is using the Nation's Report Card (NAEP) API to pull in information about adolescent academic success from the years 2001 to 2014. (grades 4, 8, 12)

In [25]:
import requests
import pandas as pd

#### Getting familiar with the API

In [2]:
endpoint = 'https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx?type=data&variable=TOTAL&subject=writing&grade=8&subscale=WRIRP&jurisdiction=NP&stattype=MN:MN&Year=2011'

response = requests.get(endpoint)

In [3]:
response

<Response [200]>

In [4]:
response.text

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","variable":"TOTAL","variableLabel":"All students","varValue":"1","varValueLabel":"All students","value":148.705469015333,"isStatDisplayable":1,"errorFlag":0}]}'

In [5]:
res = response.json()
res

{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'variable': 'TOTAL',
   'variableLabel': 'All students',
   'varValue': '1',
   'varValueLabel': 'All students',
   'value': 148.705469015333,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

In [26]:
endpoint2 = 'https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx?'

In [66]:
params = {
    'type': 'data',
    'variable': 'TOTAL', 
    'subject': 'reading',
    'grade': 4,
    'subscale': 'RRPCM',
    'jurisdiction': 'NP',
    'stattype': 'MN:MN',
    'year': 1996
}

In [67]:
response2 = requests.get(endpoint2, params = params)

In [68]:
response2.text

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'RED\' AND Cohort=1 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'1996R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1953\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1713\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 560\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 136"}'

In [62]:
res = response2.json()
res

{'status': 200,
 'result': [{'year': 1996,
   'sample': 'R3',
   'yearSampleLabel': '1996',
   'Cohort': 1,
   'CohortLabel': 'Grade 4',
   'stattype': 'MN:MN',
   'subject': 'MAT',
   'grade': 4,
   'scale': 'MRPCM',
   'jurisdiction': 'NP',
   'variable': 'TOTAL',
   'variableLabel': 'All students',
   'varValue': '1',
   'varValueLabel': 'All students',
   'value': 222.034757194769,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

In [11]:
res['result'][0]['value']

261.491530680691

### Gathering test scores for grades 4, 8, and 12 through years 1980 - 2012. All scores are composite scores. The code first tries finding the scores at a state level. If state level scores aren't found it will look for the national score. If nothing is found it will return 'N/A' for the composite score.

In [12]:
endpoint = 'https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx?'

years = [1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012 ]
grades = [4, 8, 12]
subjects = ['mathematics', 'reading', 'science', 'history', 'geography']
subscales = ['MRPCM', 'RRPCM', 'SRPCM', 'HRPCM', 'GRPCM']
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID',
          'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS',
          'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
          'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV',
          'WI', 'WY']

state = []
jurisdiction = []
year = []
grade = []
subject = []
composite_score = []

for x in years:
    for z in states:
        for w in grades:
            for y in subjects:
                print('working on', w, x, z, y)
        
                if y == 'mathematics':
                    try: 
                        params = {
                            'type': 'data',
                            'variable': 'TOTAL', 
                            'subject': y,
                            'grade': w,
                            'subscale': 'MRPCM',
                            'jurisdiction': z,
                            'stattype': 'MN:MN',
                            'year': x
                        }
        
                        response = requests.get(endpoint, params = params)
                        res = response.json()
# it's important we append 'composite_score' first just in case we get an 'empty' response back that
# doesn't provoke an error.
                        composite_score.append(res['result'][0]['value'])
                        year.append(x)
                        state.append(z)
                        jurisdiction.append('State')
                        grade.append(w)
                        subject.append(y)
        
                    except:
                        try:
                            params = {
                                'type': 'data',
                                'variable': 'TOTAL', 
                                'subject': y,
                                'grade': w,
                                'subscale': 'MRPCM',
                                'jurisdiction': 'NP',
                                'stattype': 'MN:MN',
                                'year': x
                            }
        
                            response = requests.get(endpoint, params = params)
                            res = response.json()

                            composite_score.append(res['result'][0]['value'])
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                    
                        except:
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                            composite_score.append('N/A')
                
                if y == 'reading':
                    try: 
                        params = {
                            'type': 'data',
                            'variable': 'TOTAL', 
                            'subject': y,
                            'grade': w,
                            'subscale': 'RRPCM',
                            'jurisdiction': z,
                            'stattype': 'MN:MN',
                            'year': x
                        }
        
                        response = requests.get(endpoint, params = params)
                        res = response.json()
                        
                        composite_score.append(res['result'][0]['value'])
                        year.append(x)
                        state.append(z)
                        jurisdiction.append('State')
                        grade.append(w)
                        subject.append(y)
        
                    except:
                        try:
                            params = {
                                'type': 'data',
                                'variable': 'TOTAL', 
                                'subject': y,
                                'grade': w,
                                'subscale': 'RRPCM',
                                'jurisdiction': 'NP',
                                'stattype': 'MN:MN',
                                'year': x
                            }
                           
                            response = requests.get(endpoint, params = params)
                            res = response.json()
                        
                            composite_score.append(res['result'][0]['value'])
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y) 
                        
                        except:
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                            composite_score.append('N/A')
                
                if y == 'science':
                    try: 
                        params = {
                            'type': 'data',
                            'variable': 'TOTAL', 
                            'subject': y,
                            'grade': w,
                            'subscale': 'SRPCM',
                            'jurisdiction': z,
                            'stattype': 'MN:MN',
                            'year': x
                        }
        
                        response = requests.get(endpoint, params = params)
                        res = response.json()
                        
                        composite_score.append(res['result'][0]['value'])
                        year.append(x)
                        state.append(z)
                        jurisdiction.append('State')
                        grade.append(w)
                        subject.append(y)
        
                    except:
                        try:
                            params = {
                                'type': 'data',
                                'variable': 'TOTAL', 
                                'subject': y,
                                'grade': w,
                                'subscale': 'SRPCM',
                                'jurisdiction': 'NP',
                                'stattype': 'MN:MN',
                                'year': x
                            }
        
                            response = requests.get(endpoint, params = params)
                            res = response.json()
                        
                            composite_score.append(res['result'][0]['value'])
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                            
                        except:
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                            composite_score.append('N/A')
                
                if y == 'history':
                    try: 
                        params = {
                            'type': 'data',
                            'variable': 'TOTAL', 
                            'subject': y,
                            'grade': w,
                            'subscale': 'HRPCM',
                            'jurisdiction': z,
                            'stattype': 'MN:MN',
                            'year': x
                        }
        
                        response = requests.get(endpoint, params = params)
                        res = response.json()
        
                        composite_score.append(res['result'][0]['value'])
                        year.append(x)
                        state.append(z)
                        jurisdiction.append('State')
                        grade.append(w)
                        subject.append(y)
        
                    except:
                        try:
                            params = {
                                'type': 'data',
                                'variable': 'TOTAL', 
                                'subject': y,
                                'grade': w,
                                'subscale': 'HRPCM',
                                'jurisdiction': 'NP',
                                'stattype': 'MN:MN',
                                'year': x
                            }
        
                            response = requests.get(endpoint, params = params)
                            res = response.json()
        
                            composite_score.append(res['result'][0]['value'])
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                            
                        except:
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                            composite_score.append('N/A')
                
                if y == 'geography':
                    try: 
                        params = {
                            'type': 'data',
                            'variable': 'TOTAL', 
                            'subject': y,
                            'grade': w,
                            'subscale': 'GRPCM',
                            'jurisdiction': z,
                            'stattype': 'MN:MN',
                            'year': x
                        }
        
                        response = requests.get(endpoint, params = params)
                        res = response.json()
        
                        composite_score.append(res['result'][0]['value'])
                        year.append(x)
                        state.append(z)
                        jurisdiction.append('State')
                        grade.append(w)
                        subject.append(y)
        
                    except:
                        try:
                            params = {
                                'type': 'data',
                                'variable': 'TOTAL', 
                                'subject': y,
                                'grade': w,
                                'subscale': 'GRPCM',
                                'jurisdiction': 'NP',
                                'stattype': 'MN:MN',
                                'year': x
                            }
        
                            response = requests.get(endpoint, params = params)
                            res = response.json()
        
                            composite_score.append(res['result'][0]['value'])
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                        
                        except:
                            year.append(x)
                            state.append(z)
                            jurisdiction.append('National')
                            grade.append(w)
                            subject.append(y)
                            composite_score.append('N/A')

working on 4 1980 AL mathematics
working on 4 1980 AL reading
working on 4 1980 AL science
working on 4 1980 AL history
working on 4 1980 AL geography
working on 8 1980 AL mathematics
working on 8 1980 AL reading
working on 8 1980 AL science
working on 8 1980 AL history
working on 8 1980 AL geography
working on 12 1980 AL mathematics
working on 12 1980 AL reading
working on 12 1980 AL science
working on 12 1980 AL history
working on 12 1980 AL geography
working on 4 1980 AK mathematics
working on 4 1980 AK reading
working on 4 1980 AK science
working on 4 1980 AK history
working on 4 1980 AK geography
working on 8 1980 AK mathematics
working on 8 1980 AK reading
working on 8 1980 AK science
working on 8 1980 AK history
working on 8 1980 AK geography
working on 12 1980 AK mathematics
working on 12 1980 AK reading
working on 12 1980 AK science
working on 12 1980 AK history
working on 12 1980 AK geography
working on 4 1980 AZ mathematics
working on 4 1980 AZ reading
working on 4 1980 AZ s

working on 12 1980 LA geography
working on 4 1980 ME mathematics
working on 4 1980 ME reading
working on 4 1980 ME science
working on 4 1980 ME history
working on 4 1980 ME geography
working on 8 1980 ME mathematics
working on 8 1980 ME reading
working on 8 1980 ME science
working on 8 1980 ME history
working on 8 1980 ME geography
working on 12 1980 ME mathematics
working on 12 1980 ME reading
working on 12 1980 ME science
working on 12 1980 ME history
working on 12 1980 ME geography
working on 4 1980 MD mathematics
working on 4 1980 MD reading
working on 4 1980 MD science
working on 4 1980 MD history
working on 4 1980 MD geography
working on 8 1980 MD mathematics
working on 8 1980 MD reading
working on 8 1980 MD science
working on 8 1980 MD history
working on 8 1980 MD geography
working on 12 1980 MD mathematics
working on 12 1980 MD reading
working on 12 1980 MD science
working on 12 1980 MD history
working on 12 1980 MD geography
working on 4 1980 MA mathematics
working on 4 1980 M

working on 12 1980 OK history
working on 12 1980 OK geography
working on 4 1980 OR mathematics
working on 4 1980 OR reading
working on 4 1980 OR science
working on 4 1980 OR history
working on 4 1980 OR geography
working on 8 1980 OR mathematics
working on 8 1980 OR reading
working on 8 1980 OR science
working on 8 1980 OR history
working on 8 1980 OR geography
working on 12 1980 OR mathematics
working on 12 1980 OR reading
working on 12 1980 OR science
working on 12 1980 OR history
working on 12 1980 OR geography
working on 4 1980 PA mathematics
working on 4 1980 PA reading
working on 4 1980 PA science
working on 4 1980 PA history
working on 4 1980 PA geography
working on 8 1980 PA mathematics
working on 8 1980 PA reading
working on 8 1980 PA science
working on 8 1980 PA history
working on 8 1980 PA geography
working on 12 1980 PA mathematics
working on 12 1980 PA reading
working on 12 1980 PA science
working on 12 1980 PA history
working on 12 1980 PA geography
working on 4 1980 RI m

working on 12 1984 AR science
working on 12 1984 AR history
working on 12 1984 AR geography
working on 4 1984 CA mathematics
working on 4 1984 CA reading
working on 4 1984 CA science
working on 4 1984 CA history
working on 4 1984 CA geography
working on 8 1984 CA mathematics
working on 8 1984 CA reading
working on 8 1984 CA science
working on 8 1984 CA history
working on 8 1984 CA geography
working on 12 1984 CA mathematics
working on 12 1984 CA reading
working on 12 1984 CA science
working on 12 1984 CA history
working on 12 1984 CA geography
working on 4 1984 CO mathematics
working on 4 1984 CO reading
working on 4 1984 CO science
working on 4 1984 CO history
working on 4 1984 CO geography
working on 8 1984 CO mathematics
working on 8 1984 CO reading
working on 8 1984 CO science
working on 8 1984 CO history
working on 8 1984 CO geography
working on 12 1984 CO mathematics
working on 12 1984 CO reading
working on 12 1984 CO science
working on 12 1984 CO history
working on 12 1984 CO ge

working on 12 1984 MI reading
working on 12 1984 MI science
working on 12 1984 MI history
working on 12 1984 MI geography
working on 4 1984 MN mathematics
working on 4 1984 MN reading
working on 4 1984 MN science
working on 4 1984 MN history
working on 4 1984 MN geography
working on 8 1984 MN mathematics
working on 8 1984 MN reading
working on 8 1984 MN science
working on 8 1984 MN history
working on 8 1984 MN geography
working on 12 1984 MN mathematics
working on 12 1984 MN reading
working on 12 1984 MN science
working on 12 1984 MN history
working on 12 1984 MN geography
working on 4 1984 MS mathematics
working on 4 1984 MS reading
working on 4 1984 MS science
working on 4 1984 MS history
working on 4 1984 MS geography
working on 8 1984 MS mathematics
working on 8 1984 MS reading
working on 8 1984 MS science
working on 8 1984 MS history
working on 8 1984 MS geography
working on 12 1984 MS mathematics
working on 12 1984 MS reading
working on 12 1984 MS science
working on 12 1984 MS hi

...

working on 8 2008 NE geography
working on 12 2008 NE mathematics
working on 12 2008 NE reading
working on 12 2008 NE science
working on 12 2008 NE history
working on 12 2008 NE geography
working on 4 2008 NV mathematics
working on 4 2008 NV reading
working on 4 2008 NV science
working on 4 2008 NV history
working on 4 2008 NV geography
working on 8 2008 NV mathematics
working on 8 2008 NV reading
working on 8 2008 NV science
working on 8 2008 NV history
working on 8 2008 NV geography
working on 12 2008 NV mathematics
working on 12 2008 NV reading
working on 12 2008 NV science
working on 12 2008 NV history
working on 12 2008 NV geography
working on 4 2008 NH mathematics
working on 4 2008 NH reading
working on 4 2008 NH science
working on 4 2008 NH history
working on 4 2008 NH geography
working on 8 2008 NH mathematics
working on 8 2008 NH reading
working on 8 2008 NH science
working on 8 2008 NH history
working on 8 2008 NH geography
working on 12 2008 NH mathematics
working on 12 2008 

working on 8 2008 VT history
working on 8 2008 VT geography
working on 12 2008 VT mathematics
working on 12 2008 VT reading
working on 12 2008 VT science
working on 12 2008 VT history
working on 12 2008 VT geography
working on 4 2008 VA mathematics
working on 4 2008 VA reading
working on 4 2008 VA science
working on 4 2008 VA history
working on 4 2008 VA geography
working on 8 2008 VA mathematics
working on 8 2008 VA reading
working on 8 2008 VA science
working on 8 2008 VA history
working on 8 2008 VA geography
working on 12 2008 VA mathematics
working on 12 2008 VA reading
working on 12 2008 VA science
working on 12 2008 VA history
working on 12 2008 VA geography
working on 4 2008 WA mathematics
working on 4 2008 WA reading
working on 4 2008 WA science
working on 4 2008 WA history
working on 4 2008 WA geography
working on 8 2008 WA mathematics
working on 8 2008 WA reading
working on 8 2008 WA science
working on 8 2008 WA history
working on 8 2008 WA geography
working on 12 2008 WA ma

working on 8 2012 IL science
working on 8 2012 IL history
working on 8 2012 IL geography
working on 12 2012 IL mathematics
working on 12 2012 IL reading
working on 12 2012 IL science
working on 12 2012 IL history
working on 12 2012 IL geography
working on 4 2012 IN mathematics
working on 4 2012 IN reading
working on 4 2012 IN science
working on 4 2012 IN history
working on 4 2012 IN geography
working on 8 2012 IN mathematics
working on 8 2012 IN reading
working on 8 2012 IN science
working on 8 2012 IN history
working on 8 2012 IN geography
working on 12 2012 IN mathematics
working on 12 2012 IN reading
working on 12 2012 IN science
working on 12 2012 IN history
working on 12 2012 IN geography
working on 4 2012 IA mathematics
working on 4 2012 IA reading
working on 4 2012 IA science
working on 4 2012 IA history
working on 4 2012 IA geography
working on 8 2012 IA mathematics
working on 8 2012 IA reading
working on 8 2012 IA science
working on 8 2012 IA history
working on 8 2012 IA geogr

working on 8 2012 NM reading
working on 8 2012 NM science
working on 8 2012 NM history
working on 8 2012 NM geography
working on 12 2012 NM mathematics
working on 12 2012 NM reading
working on 12 2012 NM science
working on 12 2012 NM history
working on 12 2012 NM geography
working on 4 2012 NY mathematics
working on 4 2012 NY reading
working on 4 2012 NY science
working on 4 2012 NY history
working on 4 2012 NY geography
working on 8 2012 NY mathematics
working on 8 2012 NY reading
working on 8 2012 NY science
working on 8 2012 NY history
working on 8 2012 NY geography
working on 12 2012 NY mathematics
working on 12 2012 NY reading
working on 12 2012 NY science
working on 12 2012 NY history
working on 12 2012 NY geography
working on 4 2012 NC mathematics
working on 4 2012 NC reading
working on 4 2012 NC science
working on 4 2012 NC history
working on 4 2012 NC geography
working on 8 2012 NC mathematics
working on 8 2012 NC reading
working on 8 2012 NC science
working on 8 2012 NC histo

working on 8 2012 WI mathematics
working on 8 2012 WI reading
working on 8 2012 WI science
working on 8 2012 WI history
working on 8 2012 WI geography
working on 12 2012 WI mathematics
working on 12 2012 WI reading
working on 12 2012 WI science
working on 12 2012 WI history
working on 12 2012 WI geography
working on 4 2012 WY mathematics
working on 4 2012 WY reading
working on 4 2012 WY science
working on 4 2012 WY history
working on 4 2012 WY geography
working on 8 2012 WY mathematics
working on 8 2012 WY reading
working on 8 2012 WY science
working on 8 2012 WY history
working on 8 2012 WY geography
working on 12 2012 WY mathematics
working on 12 2012 WY reading
working on 12 2012 WY science
working on 12 2012 WY history
working on 12 2012 WY geography


In [22]:
stategrade_scores_df = pd.DataFrame(year, columns=['year'])
stategrade_scores_df['grade'] = grade
stategrade_scores_df['state'] = state
stategrade_scores_df['jurisdiction'] = jurisdiction
stategrade_scores_df['subject'] = subject
stategrade_scores_df['composite_score'] = composite_score
stategrade_scores_df

,year,grade,state,jurisdiction,subject,composite_score
0,1980,4,AL,National,mathematics,N/A
1,1980,4,AL,National,reading,N/A
2,1980,4,AL,National,science,N/A
3,1980,4,AL,National,history,N/A
4,1980,4,AL,National,geography,N/A
...,...,...,...,...,...,...
6745,2012,12,WY,National,mathematics,N/A
6746,2012,12,WY,National,reading,N/A
6747,2012,12,WY,National,science,N/A
6748,2012,12,WY,National,history,N/A


In [23]:
stategrade_scores = stategrade_scores_df[stategrade_scores_df['composite_score'] != 'N/A']
stategrade_scores

,year,grade,state,jurisdiction,subject,composite_score
3000,1996,4,AL,National,mathematics,222.034757
3002,1996,4,AL,National,science,145.190133
3005,1996,8,AL,National,mathematics,268.797474
3007,1996,8,AL,National,science,147.716406
3010,1996,12,AL,National,mathematics,300.583218
...,...,...,...,...,...,...
4487,2000,4,WY,State,science,156.072713
4490,2000,8,WY,State,mathematics,275.572332
4492,2000,8,WY,State,science,156.273235
4495,2000,12,WY,National,mathematics,298.793175


In [15]:
state_math_scores = stategrade_scores[stategrade_scores['subject'] == 'mathematics']
state_science_scores = stategrade_scores[stategrade_scores['subject'] == 'science']
state_history_scores = stategrade_scores[stategrade_scores['subject'] == 'history']
state_geography_scores = stategrade_scores[stategrade_scores['subject'] == 'geography']
state_reading_scores = stategrade_scores[stategrade_scores['subject'] == 'reading']

In [16]:
state_math_scores

,year,grade,state,jurisdiction,subject,composite_score
3000,1996,4,AL,National,mathematics,222.034757
3005,1996,8,AL,National,mathematics,268.797474
3010,1996,12,AL,National,mathematics,300.583218
3015,1996,4,AK,National,mathematics,222.034757
3020,1996,8,AK,National,mathematics,268.797474
...,...,...,...,...,...,...
4475,2000,8,WI,National,mathematics,271.829035
4480,2000,12,WI,National,mathematics,298.793175
4485,2000,4,WY,State,mathematics,228.626636
4490,2000,8,WY,State,mathematics,275.572332


In [17]:
state_science_scores

,year,grade,state,jurisdiction,subject,composite_score
3002,1996,4,AL,National,science,145.190133
3007,1996,8,AL,National,science,147.716406
3012,1996,12,AL,National,science,149.783342
3017,1996,4,AK,National,science,145.190133
3022,1996,8,AK,National,science,147.716406
...,...,...,...,...,...,...
4477,2000,8,WI,National,science,147.706434
4482,2000,12,WI,National,science,145.01651
4487,2000,4,WY,State,science,156.072713
4492,2000,8,WY,State,science,156.273235


In [18]:
state_history_scores

,year,grade,state,jurisdiction,subject,composite_score


In [19]:
state_geography_scores

,year,grade,state,jurisdiction,subject,composite_score


In [20]:
state_reading_scores

,year,grade,state,jurisdiction,subject,composite_score
3751,2000,4,AL,National,reading,211.313126
3766,2000,4,AK,National,reading,211.313126
3781,2000,4,AZ,National,reading,211.313126
3796,2000,4,AR,National,reading,211.313126
3811,2000,4,CA,National,reading,211.313126
3826,2000,4,CO,National,reading,211.313126
3841,2000,4,CT,National,reading,211.313126
3856,2000,4,DE,National,reading,211.313126
3871,2000,4,FL,National,reading,211.313126
3886,2000,4,GA,National,reading,211.313126


In [24]:
stategrade_scores.to_csv('data/state_grade_composite_scores_1980_2000.csv')